In [4]:
import pandas as pd
import numpy as np

In [5]:
ctd = pd.read_csv('Customer_transactions_agg.csv')

In [6]:
t = pd.read_csv('train.csv')

In [7]:
ctd.head(5)

,customer_id,date,TotalPrice,selling_price,other_discount,coupon_discount,InvoiceNo
0,1,2/21/2012,2101.23,2603.45,-537.84,-35.62,2012-02-211
1,1,3/8/2012,1125.22,1431.20,-305.98,0.00,2012-03-081
2,1,3/29/2012,511.85,691.73,-179.88,0.00,2012-03-291
3,1,4/4/2012,1600.04,1913.50,-331.27,-17.81,2012-04-041
4,1,4/11/2012,1155.87,1702.28,-546.41,0.00,2012-04-111


In [20]:
ctd_agg = ctd.groupby(['customer_id']).agg({'TotalPrice': 'sum'})

In [21]:
ctd_agg.head()

,TotalPrice
customer_id,
1,83308.63
2,36297.42
3,59216.28
4,31135.08
5,92888.86


In [23]:
ctd_agg.shape

(1582, 1)

In [8]:
t.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [9]:
cust_coupon= t.customer_id.sort_values().unique().tolist()
print(len(cust_coupon))

1428


In [10]:
cust_transactions= ctd.customer_id.sort_values().unique().tolist()
print(len(cust_transactions))

1582


In [12]:
t_rm = t.groupby(['customer_id']).agg({'redemption_status': 'sum'})

In [13]:
t_rm.head()

,redemption_status
customer_id,
1,2
3,0
4,0
5,0
6,0


In [15]:
print(t_rm.shape)

(1428, 1)


In [24]:
ctd_t = pd.merge(ctd_agg, t_rm, on='customer_id',how='left' )

In [25]:
ctd_t.head()

,TotalPrice,redemption_status
customer_id,,
1,83308.63,2.0
2,36297.42,NaN
3,59216.28,0.0
4,31135.08,0.0
5,92888.86,0.0


In [26]:
ctd_t.loc[ctd_t['redemption_status'] >= 0, 'flag'] = 1

In [28]:
ctd_t.shape

(1582, 3)

In [29]:
ctd_t['flag']=ctd_t['flag'].fillna(0)
ctd_t.head()

,TotalPrice,redemption_status,flag
customer_id,,,
1,83308.63,2.0,1.0
2,36297.42,NaN,0.0
3,59216.28,0.0,1.0
4,31135.08,0.0,1.0
5,92888.86,0.0,1.0


In [30]:
ctd_t=ctd_t.drop(['redemption_status'],axis=1)

In [31]:
ctd_t=ctd_t.reset_index()

In [32]:
ctd_t.head()

,customer_id,TotalPrice,flag
0,1,83308.63,1.0
1,2,36297.42,0.0
2,3,59216.28,1.0
3,4,31135.08,1.0
4,5,92888.86,1.0


In [33]:
rm_final = ctd_t.groupby(['flag']).agg({'customer_id': 'count','TotalPrice':'sum'})

In [34]:
rm_final.head()

,customer_id,TotalPrice
flag,,
0.0,154,6.929663e+06
1.0,1428,1.217963e+08


In [35]:
rm_final.rename(columns = {'customer_id': 'No. of Customers'}, inplace=True)

In [36]:
rm_final.reset_index()

,flag,No. of Customers,TotalPrice
0,0.0,154,6.929663e+06
1,1.0,1428,1.217963e+08


In [38]:
rm_final['Respone_measurement']=rm_final['TotalPrice']/rm_final['No. of Customers']

In [39]:
rm_final.head()

,No. of Customers,TotalPrice,Respone_measurement
flag,,,
0.0,154,6.929663e+06,44997.811558
1.0,1428,1.217963e+08,85291.526366


In [40]:
response_measurement = (85291.526366/44997.811558)

In [41]:
response_measurement

1.8954594326451488